<a href="https://colab.research.google.com/github/elainedias16/Data-Mining/blob/main/LeanDL_HPC_Challenge_2025_Atividade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sobre o Workshop LeanDL-HPC 2025

https://sites.labic.icmc.usp.br/leandl2025/

O LeanDL-HPC 20225 se propõe a explorar e difundir técnicas que permitam **tornar o Deep Learning mais eficiente e escalável no contexto de HPC**, especialmente quando os modelos ultrapassam os recursos típicos dessas infraestruturas. Entre os tópicos em foco estão:

* **Distilação e compressão de modelos**.
* **Quantização**, **pruning**, **cálculos esparços** e **afinamento eficiente de parâmetros**.
* **Otimização de pipelines de inferência** e **deploy consciente de memória/computação**.
* Treinamento e ajuste de LLMs (Modelos de Linguagem de Grande Porte) em sistemas HPC.
* Uso de **arquiteturas leves** como GNNs e CNNs para aplicações científicas.
* Estratégias de **Green AI** e eficiência energética em DL para HPC.



## Este Notebook e o Desafio

Este notebook apresenta **detalhes sobre um dataset** preparado especialmente para um **challenge do workshop LeanDL-HPC 2025**.

O uso deste conjunto de dados é **opcional** e voltado a participantes que desejam trabalhar em uma **tarefa específica** proposta pelo evento.

O desafio envolve **mapear uma amostra teses e dissertações de 2023 para temas estratégicos predefinidos de cada UF**, explorando diferentes níveis de aderência (**BAIXA**, **MÉDIA**, **ALTA**) e, quando possível, apresentando justificativas.

Além de servir como recurso de exploração inicial, o notebook pode apoiar na preparação de soluções para submissão no workshop, alinhadas aos objetivos do LeanDL-HPC 2025.


# LEANDL 2025 — Template de Leitura de Dados e Guia do Desafio

Este notebook apresenta:
- A leitura de **dois arquivos Parquet**:
  - `leandl_oesnpg_dicionario.parquet`: **dicionário de dados** contendo a descrição dos campos.
  - `leandl_oesnpg_dados.parquet`: **dados** contendo amostras de **teses e dissertações de 2023**.
- Um **resumo técnico do desafio**.
- Diretrizes e sugestões de **abordagens de baixo custo** (classificação tradicional, BERT, GNNs, e LLMs até 14B).
- Referências para **medir custo computacional** e **pegada de carbono**.
- Observações importantes sobre a **publicidade** das soluções.



## Desafio: Mapeamento de Teses e Dissertações em Temas Estratégicos da UF

O objetivo é **mapear** cada produção acadêmica (tese/dissertação) para **temas estratégicos predefinidos** do seu estado (UF).

**Observações**:
- Uma mesma produção pode estar associada a **mais de um tema** dentro da mesma UF.
- O mapeamento deve considerar **níveis de aderência**: **BAIXA**, **MÉDIA** e **ALTA**.
- A **justificativa do mapeamento** é **relevante** (explica a decisão) e **bem-vinda**, mas **não é obrigatória**.



## Leitura dos Arquivos Parquet

Abaixo, o código para carregar os dois arquivos:
- `leandl_oesnpg_dicionario.parquet` (dicionário de dados);
- `leandl_oesnpg_dados.parquet` (dados principais).


In [1]:
!pip install codecarbon # reiniciar apos instalar este pacote se você estiver em um notebook

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)


In [37]:
import pandas as pd

#baixando os arquivos (instalar o pacote gdown)
#!gdown 12H957uf6mK-1X_ztT9hgFS1slpN2j-Wh
#!gdown 1-QXkqH8HzLcV2JCA4Nm9G5rQhorYKJVe

# Ajuste os caminhos se necessário
path_dict = "../elaine/leandl_oesnpg_dicionario.parquet"
path_data = "../elaine/leandl_oesnpg_dados.parquet"

# Leitura usando pandas (requer pyarrow ou fastparquet)
dicionario_df = pd.read_parquet(path_dict)
dados_df = pd.read_parquet(path_data)

print("Dimensões do dicionário:", dicionario_df.shape)
print("Dimensões dos dados:", dados_df.shape)




Dimensões do dicionário: (25, 2)
Dimensões dos dados: (42046, 25)


In [38]:
campo = dicionario_df['campo'].tolist()
campo

['hash_id',
 'tema_id',
 'tema',
 'palavras_chave',
 'uf_tema_info',
 'uf_pesquisador',
 'nome_programa',
 'sigla_entidade_ensino',
 'nome_producao',
 'nome_subtipo_producao',
 'nome_area_concentracao',
 'nome_linha_pesquisa',
 'nome_projeto',
 'descricao_palavra_chave',
 'descricao_resumo',
 'descricao_abstract',
 'descricao_keyword',
 'data_titulacao',
 'nome_grau_academico',
 'nome_grande_area_conhecimento',
 'nome_area_conhecimento',
 'nome_subarea_conhecimento',
 'nome_especialidade',
 'modelo_nivel',
 'modelo_explicacao']

In [39]:
primeiro_valor = dados_df['modelo_explicacao'].iloc[0]
print(primeiro_valor)

A afinidade entre os dados do pesquisador e o tema estratégico é classificada como "BAIXA". Embora o pesquisador trabalhe com "tecnologias digitais de informação e comunicação" no contexto educacional, o tema estratégico foca no "Agronegócio e Tecnologias de Informação e Comunicação". A única correlação temática é a menção às "tecnologias da informação e comunicação (TIC)", que aparece em ambos os contextos, mas o foco do pesquisador é na educação, enquanto o tema estratégico está voltado para o agronegócio e suas aplicações tecnológicas. Não há menção a elementos específicos do agronegócio, como "agroindústria", "cadeias produtivas" ou "sustentabilidade", nos dados do pesquisador.


In [40]:
# dados_df["tema"]
"Combate às Iniquidades" in dados_df["tema"].values


False

## Relação de Temas e suas Palavras-Chave, Nome-Producao, Descricao-Palavra-Chave, Descricao-resumo

In [5]:
# import numpy as np
# import pandas as pd

# def to_fset(x):
#     if isinstance(x, (list, tuple, np.ndarray)):
#         return frozenset(x)
#     if pd.isna(x):
#         return frozenset()
#     return frozenset([x])

# df_temas = (
#     dados_df.assign(palavras_chave=dados_df["palavras_chave"].apply(to_fset))
#             [["tema_id", "tema", "uf_tema_info", "nome_producao", "palavras_chave", "descricao_palavra_chave","descricao_resumo", "modelo_explicacao" ]]
#             .drop_duplicates()
# )

# df_temas


## Níveis de Aderência (BAIXA, MÉDIA, ALTA)

Atribua **níveis de aderência** que reflitam o grau de correspondência entre o **tema estratégico**
e o **conteúdo** da produção (título, resumo, palavras-chave, etc.).

Sugestão de interpretação:
- **ALTA**: forte coerência semântica e conceitual; termos-chave do tema presentes de forma central no trabalho.
- **MÉDIA**: relação indireta ou parcial; termos aparecem com relevância moderada ou secundária.
- **BAIXA**: relação fraca; termos aparecem superficialmente ou em contextos distintos do tema.

Caso produza **explicações** (ex.: `modelo_explicacao`), procure referenciar **trechos** e **termos** que sustentem a decisão.



## Abordagens de Baixo Custo Recomendadas

### 1) Classificadores tradicionais / BERT (binário: **ALTO** vs **NÃO-ALTO**)
- **Embeddings** de título + resumo (ex.: SBERT, sentence-transformers).
- **Similaridade** com descrições dos temas (cosine similarity).
- **k-NN** ou **regressão logística** (binário) para decidir se é **ALTO** ou **NÃO-ALTO**.
- **Fine-tuning BERT** com amostras rotuladas (quando disponíveis), restringindo o problema a **ALTO** vs **NÃO-ALTO** para reduzir custo.

### 2) LLMs até **14B** de parâmetros (com ou sem fine-tuning)
- **Zero-shot** e **few-shot** com **avaliação sistemática de prompts**.
- **Quantização** (ex.: 4-bit) para reduzir memória/latência.
- **Instruções** claras: definir o papel do modelo, a rubrica de decisão (BAIXA/MÉDIA/ALTA) e pedidos de explicação concisa.
- **Auto-avaliação** de confiança e calibragem (ex.: pedir ao modelo que dê um nível e uma justificativa, depois verificar consistência).

### 3) Estratégias baseadas em grafos
- Modelagem de **grafos de similaridade** entre produções e temas.
- **GNNs** (Graph Neural Networks) para propagar sinais de rótulo e robustecer decisões.
- Uso de **k-NN em embeddings** para construir arestas e explorar **comunidades** temáticas.



## Avaliação

As soluções submetidas devem vir com uma **descrição técnica** (preferencialmente um artigo submtido na main track do workshop LeanDL) cobrindo:
1. **Acurácia do mapeamento** por nível (BAIXA/MÉDIA/ALTA) ou na formulação binária (ALTO vs NÃO-ALTO).
2. **Custo computacional**: **tempo** e **memória** consumidos durante inferência/treinamento.
3. (Opcional) **Matriz de confusão** e análises de erro para interpretar decisões.

OBS: quem preferir apresentar em formato de poster, deve submeter apenas um abstract detalhando sua solução e uma cópia do código.

### Medindo Pegada de Carbono
Sugestão: utilizar a biblioteca **codecarbon**.
- Instalação e instruções: https://mlco2.github.io/codecarbon/installation.html



## Observações Finais

- **Origem dos dados**: todas as informações são públicas, obtidas do Portal de Transparência da CAPES e do Observatório da Pós-Graduação da CAPES, com foco em **amostras de teses e dissertações defendidas em 2023**.
- **Publicidade das soluções**: as soluções propostas devem ser **públicas** para garantir a **avaliação** por membros do comitê científico do LeanDL-HPC 2025.


# Exemplo simplificado (apenas para ilustração)

Vamos carregar um modelo de linguagem.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-4B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Usando ZeroShot para classificar em algum tema estratégico!

In [7]:
from codecarbon import EmissionsTracker # para calcular emissões de CO2
tracker = EmissionsTracker()
tracker.start()

# este é um prompt apenas ilustrativo!
prompt = """Você é um avaliador de aderência temática.
Classifique a afinidade entre uma produção acadêmica (título e resumo)
e um TEMA ESTRATÉGICO.

TÍTULO: Heterogeneous Graphs for Text Representation: An Integrated Approach with Language Models

PALAVRAS-CHAVE:
- Heterogeneous networks
- Language models
- Opinion mining
- Representation learning
- Unified embedding space

RESUMO:
Data representation through graphs is essential for analyzing complex relationships in fields like computer science and biology. In real-world scenarios, relationships between vertices do not always follow a uniform pattern, creating the need for heterogeneous graphs representing different types of vertices and various relationships in complex systems. However, heterogeneous graphs come with challenges. Due to the diversity of vertices and types of relationships, the inherent complexity of these structures makes understanding and analyzing them more complex than homogeneous graphs. To address this challenge, several machine learning models specific to heterogeneous graphs have been developed to comprehend the semantics of relationships between entities. Text representation in heterogeneous graphs is also challenging due to the lack of structure in textual data, which can lead to information loss. Additionally, heterogeneous graphs struggle to capture detailed semantic information in texts as they are primarily designed to represent formal structures and structural relationships. Resolving textual ambiguities is also complex for heterogeneous graphs, requiring a deep understanding of textual context. While language models excel at text comprehension, they may not be suitable for representing complex entities and relationships in real-world systems. Accurately identifying entities mentioned in texts and their relationships with real-world entities can be challenging. The integration of heterogeneous graphs and language models offers a promising solution. It combines the structural knowledge of heterogeneous graphs with the textual understanding of language models, resulting in embeddings that incorporate both the structural complexity of graphs and natural language text understanding. This approach can enhance performance in natural language processing, recommendation, and information retrieval tasks. This doctoral thesis focuses on overcoming the limitations of heterogeneous graphs in representing semantic information in texts. The proposal is to combine heterogeneous graphs with language models, leveraging the advantages of both approaches. While graphs represent structures and relationships, language models specialize in efficiently understanding and generating text. The underlying hypothesis is that this combination will result in richer data representations, improving performance in complex data analyses. This thesis introduces a two-stage approach that combines label propagation techniques and language model embeddings to generate vector representations of vertices in heterogeneous graphs. In this approach, the EPHG-CR (Embedding Propagation for Heterogeneous Graphs with Class Refinement) method is proposed, which differentiates itself by considering not only edge weights but also vertex relevance to task classes, bringing vertices with the same class closer together, taking into account the graphs topology. This approach was compared with a language model in the aspect-based sentiment analysis task, showing competitive results and, in some cases, slight superiority. Furthermore, the article explores applications of auxiliary vertex embeddings in other tasks, demonstrating another advantage of the approach.

Responda, qual o NÍVEL DE AFINIDADE (ALTA, MÉDIA ou BAIXA) com o tema estratégico: "Ciência de Dados e Inteligência Artificial" ?

CRITÉRIOS GERAIS:
- ALTA: o tema é central na pesquisa; forte coerência semântica.
- MÉDIA: relação parcial/indireta ou secundária ao tema.
- BAIXA: relação fraca ou tangencial; tema não é foco do trabalho.

RESPOSTA:
"""

# https://huggingface.co/Qwen/Qwen3-4B
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("[LLM OUTPUT] thinking content:", thinking_content)
print("[LLM OUTPUT] content:", content)


emissions: float = tracker.stop()
print("\n\nTotal de emissões (detalhes em emissions.csv): ",emissions)


[codecarbon WARNING @ 18:24:45] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 18:24:45] [setup] RAM Tracking...
[codecarbon INFO @ 18:24:45] [setup] CPU Tracking...
[codecarbon WARNING @ 18:24:46] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 18:24:46] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 18:24:46] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 18:24:46] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 18:24:46] [setup] GPU Tracking...
[codecarbon INFO @ 18:24:46] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:24:46] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation mode

[LLM OUTPUT] thinking content: <think>
Okay, let's tackle this. The user wants me to determine the level of affinity between the given academic work and the strategic theme "Ciência de Dados e Inteligência Artificial" (Data Science and Artificial Intelligence). The answer should be either ALTA, MÉDIA, or BAIXA.

First, I need to understand the content of the thesis. The title is about using heterogeneous graphs for text representation with language models. The keywords include heterogeneous networks, language models, opinion mining, representation learning, and unified embedding space. The abstract talks about challenges in representing text using heterogeneous graphs, combining them with language models to improve performance in NLP tasks like sentiment analysis and information retrieval.

Now, the strategic theme is Data Science and AI. So, I need to check how much the thesis aligns with that. Data Science involves data representation, machine learning models, and using data structur

# Considerações Finais para Participação na Challenge do LeanDL-HPC 2025

* **Como artigo regular:** Se você deseja participar da challenge como **artigo regular** da conferência, submeta seu trabalho na **trilha principal (main track)** da conferência, seguindo as diretrizes do sistema de submissão.

* **Como pôster:** Se você deseja participar da challenge no **formato de pôster**, submeta um **documento de até 2 páginas** descrevendo a solução técnica (objetivo, abordagem, arquitetura/modelos, dados, avaliação e limitações).

> **Observação:** A comissão poderá solicitar, para fins de premiação, que um **novo conjunto de teste** seja executado **antes do evento**, de modo a permitir **comparação justa** entre as soluções.


## Solução Proposta

### Criando DataFrame

In [61]:
import numpy as np
import pandas as pd

def to_fset(x):
    if isinstance(x, (list, tuple, np.ndarray)):
        return frozenset(x)
    if pd.isna(x):
        return frozenset()
    #return frozenset([x])
    return frozenset(x)

df_temas = (
    dados_df.assign(palavras_chave=dados_df["palavras_chave"].apply(to_fset))
            [["tema_id", "tema", "uf_tema_info", "nome_producao", "palavras_chave", "descricao_palavra_chave","descricao_resumo", "modelo_nivel" ]]
            .drop_duplicates()
            .dropna()
)


df_temas

,tema_id,tema,uf_tema_info,nome_producao,palavras_chave,descricao_palavra_chave,descricao_resumo,modelo_nivel
0,1,Agronegócio e Tecnologias de Informação e Comu...,ACRE,AS TECNOLOGIAS DIGITAIS DA INFORMAÇÃO E COMUNI...,"(cadeias produtivas, inteligência artificial, ...",TECNOLOGIAS DA INFORMAÇÃO E COMUNICAÇÃO;BASE N...,"A PESQUISA, DESCRITA NO PRESENTE TRABALHO, ENV...",BAIXA
1,1,Agronegócio e Tecnologias de Informação e Comu...,ACRE,O CURRÍCULO INTEGRADO DO INSTITUTO FEDERAL DO ...,"(cadeias produtivas, inteligência artificial, ...",ENSINO MÉDIO INTEGRADO;ORGANIZAÇÃO CURRICULAR;...,ESTE ESTUDO INVESTIGOU O CURRÍCULO DO ENSINO M...,MEDIA
2,3,Biodiversidade e Biotecnologia,ACRE,TENDÊNCIA TEMPORAL E DISTRIBUIÇÃO ESPACIAL DAS...,"(cadeias produtivas, turismo de natureza, fome...",EPIDEMIOLOGIA;LEISHMANIA;PRAIS-WINSTEN,"INTRODUÇÃO. AS ANÁLISES NOS BIOMAS: AMAZÔNIA, ...",BAIXA
3,3,Biodiversidade e Biotecnologia,ACRE,MODELAGEM DE BIOMASSA FLORESTAL E CÁLCULO DE C...,"(cadeias produtivas, turismo de natureza, fome...",GED;SENSORIAMENTO REMOTO;FLORESTA;MAPEAMENTO;B...,DURANTE AS ÚLTIMAS DÉCADAS AS FLORESTAS TROPIC...,MEDIA
4,3,Biodiversidade e Biotecnologia,ACRE,ANÁLISE SOCIOECONÔMICA E AMBIENTAL DA CADEIA P...,"(cadeias produtivas, turismo de natureza, fome...",BURITI;PRODUTO FLORESTAL NÃO-MADEIEREIRO;COOPE...,O BIOMA AMAZÔNICO OCUPA POSIÇÃO DE DESTAQUE NO...,ALTA
...,...,...,...,...,...,...,...,...
42039,466,Sistemas Agrícolas Sustentáveis,TOCANTINS,AVALIAÇÃO ECOTOXICOLÓGICA DE EFLUENTE DE ABATE...,"(cadeias produtivas, redução das desigualdades...",FERTILIZAÇÃO.;INVERTEBRADO DE ÁGUA DOCE;INVERT...,O DESCARTE DE RESÍDUOS EM MATADOUROS É UM DESA...,MEDIA
42040,466,Sistemas Agrícolas Sustentáveis,TOCANTINS,ANÁLISES MORFO-FISIOLÓGICAS DE PLANTAS DE ACRO...,"(cadeias produtivas, redução das desigualdades...",MACAÚBA;TOXIDADE;CHIQUIMATO;HERBICIDA;IRGA,AS CULTURAS OLEAGINOSAS SÃO AS MAIS ESTUDADAS ...,BAIXA
42042,467,Transformação Digital,TOCANTINS,DESENVOLVIMENTO DE JOGO EDUCATIVO SOBRE PROPRI...,"(redes colaborativas, redução das desigualdade...",JOGO DE TABULEIRO;PROPRIEDADE INTELECTUAL;TRAN...,TRATA-SE DE UM PROTÓTIPO PARA A CONCEPÇÃO DE U...,BAIXA
42043,467,Transformação Digital,TOCANTINS,DESAFIOS E OPORTUNIDADES DA IMPLEMENTAÇÃO DA I...,"(redes colaborativas, redução das desigualdade...",INOVAÇÃO ABERTA;MARCO LEGAL DE CIÊNCIA;TECNOLO...,A INOVAÇÃO ABERTA (IA) TEM SIDO UTILIZADA COMO...,BAIXA


In [62]:
df_temas['modelo_nivel'].isna().any()
df_temas[df_temas['modelo_nivel'].isna()]


,tema_id,tema,uf_tema_info,nome_producao,palavras_chave,descricao_palavra_chave,descricao_resumo,modelo_nivel


## Pré processamento

In [6]:
import nltk
nltk.download('punkt')

!pip install unidecode
import unidecode

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('portuguese'))

from nltk.tokenize import word_tokenize

from nltk.stem import RSLPStemmer
nltk.download('rslp')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/elainedias16/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/elainedias16/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     /home/elainedias16/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/elainedias16/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
# stemming
def stemming(text):
  stemmer = nltk.stem.RSLPStemmer() # stemming to portuguese
  tokens = word_tokenize(text) #get tokens

  doc_text_stems = [stemmer.stem(i) for i in tokens]
  return doc_text_stems

def remove_stop_words(tokens):
  text = [token for token in tokens if token not in stop_words]
  return text

def remove_non_alphanumeric(tokens):
  text = [token for token in tokens if token.isalnum()]
  return text


In [43]:
def preprocessing(text):

    text = text.lower()
    text = unidecode.unidecode(text) #remove accents
    text = stemming(text) #stemming returns a vector of tokens
    text = remove_stop_words(text)
    text = remove_non_alphanumeric(text)
    return ' '.join(text)


In [64]:
def preprocess_dataset(df, columns):
  df[columns] = df[columns].astype(str).applymap(preprocessing)
  return df

In [65]:
#columns_to_preprocess = ["tema", "palavras_chave", "nome_producao", "descricao_palavra_chave", "descricao_resumo"]
columns_to_preprocess = ["tema", "descricao_resumo" ]

df_temas = preprocess_dataset(df_temas, columns_to_preprocess)
df_temas.head()

,tema_id,tema,uf_tema_info,nome_producao,palavras_chave,descricao_palavra_chave,descricao_resumo,modelo_nivel
0,1,agronegoci tecnolog informaca comunicaca,ACRE,AS TECNOLOGIAS DIGITAIS DA INFORMAÇÃO E COMUNI...,"(cadeias produtivas, inteligência artificial, ...",TECNOLOGIAS DA INFORMAÇÃO E COMUNICAÇÃO;BASE N...,pesquis descrit pres trabalh envolv doi aspect...,BAIXA
1,1,agronegoci tecnolog informaca comunicaca,ACRE,O CURRÍCULO INTEGRADO DO INSTITUTO FEDERAL DO ...,"(cadeias produtivas, inteligência artificial, ...",ENSINO MÉDIO INTEGRADO;ORGANIZAÇÃO CURRICULAR;...,est estud investig curricul ensin medi integr ...,MEDIA
2,3,biodivers biotecnolog,ACRE,TENDÊNCIA TEMPORAL E DISTRIBUIÇÃO ESPACIAL DAS...,"(cadeias produtivas, turismo de natureza, fome...",EPIDEMIOLOGIA;LEISHMANIA;PRAIS-WINSTEN,introduca analis biom amazon caating cerr mat ...,BAIXA
3,3,biodivers biotecnolog,ACRE,MODELAGEM DE BIOMASSA FLORESTAL E CÁLCULO DE C...,"(cadeias produtivas, turismo de natureza, fome...",GED;SENSORIAMENTO REMOTO;FLORESTA;MAPEAMENTO;B...,dur ult dec florest tropic sid const ameac pel...,MEDIA
4,3,biodivers biotecnolog,ACRE,ANÁLISE SOCIOECONÔMICA E AMBIENTAL DA CADEIA P...,"(cadeias produtivas, turismo de natureza, fome...",BURITI;PRODUTO FLORESTAL NÃO-MADEIEREIRO;COOPE...,biom amazon ocup posica destaqu cenari nacion ...,ALTA


## Criação dos embeddings

#pre processing took 3min

In [66]:
#Embeddings

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

In [67]:
embeddings_tema = model.encode(df_temas["tema"].tolist(), show_progress_bar=True)
embeddings_descricao_resumo = model.encode(df_temas["descricao_resumo"].tolist(), show_progress_bar=True)
print(embeddings_tema.shape)
print(embeddings_descricao_resumo.shape)

Batches: 100%|██████████| 1030/1030 [00:18<00:00, 55.92it/s]


(32931, 384)
(32931, 384)


In [68]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = []

for i in range(len(embeddings_tema)):
    sim = cosine_similarity(
        embeddings_tema[i].reshape(1, -1),  # garante que seja 2D
        embeddings_descricao_resumo[i].reshape(1, -1)
    )
    similarities.append(sim[0][0])  # extrai o valor escalar da matriz 1x1

# Converte para DataFrame
df_similarity = pd.DataFrame(similarities, columns=[ "similarity"])

df_similarity.head()

,similarity
0,0.484893
1,0.490934
2,0.281544
3,0.168180
4,0.230774


In [69]:
df_similarity.shape

(32931, 1)

In [70]:
df_temas.shape

(32931, 8)

## Criando o classificador

In [83]:
y_train

9853    NaN
7177    NaN
29340   NaN
21135   NaN
10676   NaN
         ..
21212   NaN
40895   NaN
12779   NaN
36215   NaN
13700   NaN
Name: modelo_nivel, Length: 26344, dtype: float64

In [81]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

X = df_similarity
y_name = df_temas['modelo_nivel']


y = y_name.map({'BAIXO': 0, 'MÉDIO': 1, 'ALTO': 2})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

ValueError: Input y contains NaN.

In [91]:
y_name = df_temas['modelo_nivel']

y = y_name.map({'BAIXO': 0, 'MÉDIO': 1, 'ALTO': 2})

In [94]:
y_name


y = y.astype(int)


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [75]:
X = df_similarity
y_name = df_temas['modelo_nivel']
y_name.isna().sum()
X.isna().sum()


similarity    0
dtype: int64

In [ ]:
# ### 1) Classificadores tradicionais / BERT (binário: **ALTO** vs **NÃO-ALTO**)
# - **Embeddings** de título + resumo (ex.: SBERT, sentence-transformers).
# - **Similaridade** com descrições dos temas (cosine similarity).
# - **k-NN** ou **regressão logística** (binário) para decidir se é **ALTO** ou **NÃO-ALTO**.
# - **Fine-tuning BERT** com amostras rotuladas (quando disponíveis), restringindo o problema a **ALTO** vs **NÃO-ALTO** para reduzir custo.